# Rough

In [1]:
import torch
from diffusers import FluxPipeline
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
pipe.to("cuda")
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()
pipe.to(torch.float16)

/root/miniconda3/envs/image/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:05<00:00,  1.30it/s]


FluxPipeline {
  "_class_name": "FluxPipeline",
  "_diffusers_version": "0.30.3",
  "_name_or_path": "black-forest-labs/FLUX.1-schnell",
  "scheduler": [
    "diffusers",
    "FlowMatchEulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "FluxTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [2]:
# import torch
# from diffusers import FluxPipeline

# # Create the pipeline
# pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
# pipe.to("cuda")
# # Enable CPU offloading to save VRAM (remove this line if you have enough GPU memory)
# pipe.enable_model_cpu_offload()



In [5]:

# Set up your prompt
# prompt = "A graph of normal distribution. The y-axis is given a name f(x) and x-axis is given the name 'X'. The scale of x-axis is between -1 to +1"
prompt = "The graph of projection of Y vector onto the columnspace of X in Linear regression. The image should contain a vector with name Y, a plane with name X and another vector y_bar in the plane X."

# Generate the image
image = pipe(
    prompt,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=256,
    generator=torch.Generator("cuda").manual_seed(0)
).images[0]

# Save the generated image
image.save("flux_sample2.png")

100%|██████████| 4/4 [00:24<00:00,  6.17s/it]


# Gradio Demo

In [1]:
import gradio as gr
import os
from together import Together
api_key = os.getenv("together_api_key")
client = Together(api_key=api_key)
import torch
from diffusers import FluxPipeline
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
pipe.to("cuda")
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()
pipe.to(torch.float16)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


FluxPipeline {
  "_class_name": "FluxPipeline",
  "_diffusers_version": "0.30.3",
  "_name_or_path": "black-forest-labs/FLUX.1-schnell",
  "scheduler": [
    "diffusers",
    "FlowMatchEulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "FluxTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [2]:

def get_text_description(input):

    prompts = f'''
    You are University level professor with extensive knowledge.
    Your task is to give a brief and appropriate description of the user's input topic, by breaking down into simpler terms.

    Prioritize clarity and brevity. Your response must not exceed 250 words.

    User input: {input}
    Output:
    '''

    client = Together(api_key=api_key)
    response = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[{"role": "user", "content": prompts}],
    )

    # Extract the selected option from the response
    response = response.choices[0].message.content.strip()
    return response

def get_text_instructions(input):

    prompts = f'''
    You are a mathematical visualization expert. Create concise, precise instructions for generating a mathematical image based on the user's input.
    
        Include only:
        1. Graph type: Specify the exact type of graph.
        2. Essential equation(s): Provide the main formula(s) to plot, ```DATA POINTS TO BE PLOTTED.```
        3. Axis labels and ranges: Use standard terms (x, y, z, t) and specify exact numerical ranges.
        4. Key features: List only critical points, intersections, or regions crucial to understanding the graph.
        5. Color scheme: Suggest up to two colors for clarity, if necessary.

        Omit any explanatory text. Use mathematical notation where appropriate. Limit your response to 100 words.

        User input: {input}
        Output:

    '''

    client = Together(api_key=api_key)
    response = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[{"role": "user", "content": prompts}],
    )

    # Extract the selected option from the response
    response = response.choices[0].message.content.strip()
    return response


def image_gen_fxn(input_prompt):
    input_prompt = f'{input_prompt} with the axis plotted and labelled.'
    image = pipe(
        input_prompt,
        guidance_scale=0.0,
        num_inference_steps=4,
        max_sequence_length=256,
        generator=torch.Generator("cuda").manual_seed(0)
        ).images[0]

    return image


def generate_text_and_image(user_input):

    text_output = get_text_description(input=user_input)
    # text_instructions = get_text_instructions(input=user_input)
    image_output = image_gen_fxn(input_prompt=user_input)

    return text_output, image_output

In [3]:
# # Create Gradio interface
# with gr.Blocks(theme="default") as iface:
#     gr.Markdown("# EduGraphix")
#     gr.Markdown("What are you exploring today? Let us enhance your journey with visual aids!")
    
#     with gr.Row():
#         with gr.Column(scale=1):
#             input_text = gr.Textbox(label="Enter a prompt", lines=5)
#             # Small Submit Button below the input text box
#             submit_button = gr.Button("Generate", elem_id="submit-button", size="small")
#         with gr.Column(scale=1):
#             text_output = gr.Textbox(label="Textual Description", lines=5)
    
#     with gr.Row():
#         with gr.Column(scale=1, min_width=0):
#             gr.Markdown("")  # Empty column for spacing
#         with gr.Column(scale=2):
#             image_output = gr.Image(label="Generated Image")
#         with gr.Column(scale=1, min_width=0):
#             gr.Markdown("")  # Empty column for spacing
    
#     # Set up the button action
#     submit_button.click(generate_text_and_image, inputs=input_text, outputs=[text_output, image_output])

# # Add custom CSS to style the button smaller
# iface.css = """
# #submit-button {
#     padding: 5px 10px; /* Adjust padding for smaller size */
#     font-size: 12px;   /* Smaller font size */
# }
# """

# # Launch the interface
# iface.launch(share=True)


# Create Gradio interface
with gr.Blocks(theme="default") as iface:
    gr.Markdown("# EduGraphix")
    gr.Markdown("What are you exploring today? Let us enhance your journey with visual aids!")
    
    with gr.Row():
        with gr.Column(scale=1):
            input_text = gr.Textbox(label="Enter a prompt", lines=5)
            submit_button = gr.Button("Generate", elem_id="submit-button", size="sm")
            text_output = gr.Textbox(label="Textual Description", lines=5)
        with gr.Column(scale=1):
            image_output = gr.Image(label="Generated Image")
    
    # Set up the button action
    submit_button.click(generate_text_and_image, inputs=input_text, outputs=[text_output, image_output])

# Add custom CSS to style the button smaller
iface.css = """
#submit-button {
    padding: 5px 10px;
    font-size: 12px;
}
"""
iface.launch(share=True)


Running on local URL:  http://0.0.0.0:6006
Running on public URL: https://65d5c5ccb41a05e12a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]